<a href="https://colab.research.google.com/github/titouan-ux/DATA_BCG/blob/main/XGboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade xgboost

import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

from sklearn.model_selection import train_test_split


In [3]:
df_train=pd.read_csv("Train.csv")

df_train=df_train.drop(columns='Unnamed: 0')
df_train_modified=df_train[['year','id arc', 'id noeud amont',
       'id noeud aval', 'etat trafic', 'etat arc', 'Temp', 'Pluie 1h',
       'Pluie 3h', 'Neige', 'Visibilité Horiz', 'cos_hour', 'sin_hour',
       'cos_day', 'sin_day', 'cos_month', 'sin_month', 'débit horaire',
       'taux occupation']]

X = df_train_modified.iloc[:, :-2]
y1 = df_train_modified[["débit horaire"]]
y2 = df_train_modified[["taux occupation"]]


X_train, X_test, y1_train, y1_test = train_test_split(X, y1, test_size= 0.2, random_state=1)
X_train, X_test, y2_train, y2_test = train_test_split(X, y2, test_size= 0.2, random_state=1)

In [4]:
import xgboost as xgb

# Create regression matrices
dtrain_reg = xgb.DMatrix(X_train, y1_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y1_test, enable_categorical=True)

In [5]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}

n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:03:36] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [6]:
from sklearn.metrics import mean_squared_error

preds = model.predict(dtest_reg)

rmse = mean_squared_error(y1_test, preds, squared=False)

print(f"RMSE of the base model: {rmse:.3f}")

RMSE of the base model: 99.764


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:03:49] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
